In [1]:
# Import required packages
import os
import sys
#import asf_search as asf
import logging
import isce
root_logger = logging.getLogger()
root_logger.setLevel('WARNING')
import numpy as np
import matplotlib.pyplot as plt
from osgeo import gdal
from datetime import datetime, timedelta
import time
from glob import glob
import asf_search as asf

In [2]:
# Set environment variables to call ISCE from the command line
os.environ['ISCE_HOME'] = os.path.dirname(isce.__file__)
os.environ['ISCE_ROOT'] = os.path.dirname(os.environ['ISCE_HOME'])
os.environ['ISCE_STACK'] = '/mnt/Backups/gbrench/sw/insar_tools/isce2/src/isce2/contrib/stack'
os.environ['PYTHONPATH'] = os.environ['ISCE_STACK']
os.environ['PATH'] += f":{os.environ['ISCE_STACK']}/topsStack"

In [3]:
scene_list=[
'S1A_IW_SLC__1SDV_20191004T121345_20191004T121412_029309_0354B0_F48E',
'S1A_IW_SLC__1SDV_20191016T121345_20191016T121412_029484_035ABB_373F',
'S1A_IW_SLC__1SDV_20191028T121345_20191028T121412_029659_0360C3_574E',
'S1A_IW_SLC__1SDV_20191109T121345_20191109T121412_029834_0366F3_462C',
'S1A_IW_SLC__1SDV_20191121T121344_20191121T121411_030009_036CFD_787C',
'S1A_IW_SLC__1SDV_20191203T121344_20191203T121411_030184_03730F_4C1A',
'S1A_IW_SLC__1SDV_20191215T121344_20191215T121411_030359_03791A_4B57',
'S1A_IW_SLC__1SDV_20191227T121343_20191227T121410_030534_037F23_3C47',
'S1A_IW_SLC__1SDV_20200108T121343_20200108T121410_030709_038533_C53D',
'S1A_IW_SLC__1SDV_20200120T121342_20200120T121409_030884_038B59_8828',
'S1A_IW_SLC__1SDV_20200201T121342_20200201T121409_031059_039174_1908',
'S1A_IW_SLC__1SDV_20200213T121341_20200213T121408_031234_039785_41EB',
'S1A_IW_SLC__1SDV_20200225T121341_20200225T121408_031409_039D91_8F87',
'S1A_IW_SLC__1SDV_20200308T121341_20200308T121408_031584_03A39B_6624',
'S1A_IW_SLC__1SDV_20200320T121342_20200320T121409_031759_03A9BC_08F7',
'S1A_IW_SLC__1SDV_20200401T121342_20200401T121409_031934_03AFE6_E721',
'S1A_IW_SLC__1SDV_20200413T121342_20200413T121409_032109_03B610_2EA6'
           ]

In [4]:
proc_path = '/mnt/Backups/gbrench/repos/fusits/nbs/imja/agu_push/AT12'

In [ ]:
# download slcs
results = asf.granule_search(scene_list)
results.download(path=f'{proc_path}/slc', processes=2)

In [ ]:
# # download dem
# os.chdir(f'{proc_path}/dem')
# !sardem --bbox 85.9 27.1 87.6 28.6 --xrate 3 --yrate 3 --data-source COP -isce

## first run to CSLCs

In [ ]:
# stacksentinel to generate run files
os.chdir(proc_path)
!stackSentinel.py -s ./slc -o ./orbits -a ./aux -d ./dem/elevation.dem -w ./work -C geometry --bbox '27.3 28.3 86.1 87.4' -W slc -C geometry --num_proc 4

In [ ]:
%%time
!sh ./work/run_files/run_01_unpack_topo_reference

In [ ]:
%%time
!sh ./work/run_files/run_02_unpack_secondary_slc

In [ ]:
%%time
!sh ./work/run_files/run_03_average_baseline

In [ ]:
%%time
!sh ./work/run_files/run_04_fullBurst_geo2rdr

In [ ]:
%%time
!sh ./work/run_files/run_05_fullBurst_resample

In [ ]:
%%time
!sh ./work/run_files/run_06_extract_stack_valid_region

In [ ]:
%%time
!sh ./work/run_files/run_07_merge_reference_secondary_slc

In [ ]:
%%time
!sh ./work/run_files/run_08_grid_baseline

In [ ]:
# remove run files
!rm -r ./work/run_files

In [ ]:
!mv ./work/coreg_secondarys ./work/coreg_secondarys_tmp
!mv ./work/merged ./work/merged_cslcs

## run to wrapped interferograms 

In [ ]:
# stacksentinel to generate run files
# NOTE: do not change work directory name to anything other than "work" or things will break
os.chdir(proc_path)
!stackSentinel.py -s ./slc -o ./work/orbits -a ./aux -d ./dem/elevation.dem -w ./work -C geometry --bbox '27.3 28.3 86.1 87.4' -W interferogram -C geometry --num_proc 4 -c 5 -z 1 -r 5  

In [ ]:
!mv ./work/coreg_secondarys_tmp ./work/coreg_secondarys

In [ ]:
%%time
!sh ./work/run_files/run_07_merge_reference_secondary_slc

In [ ]:
%%time
!sh ./work/run_files/run_08_generate_burst_igram

In [ ]:
%%time
!sh ./work/run_files/run_09_merge_burst_igram

In [ ]:
%%time
!sh ./work/run_files/run_10_filter_coherence